### Info

Author: Jelle Poland \
Citing: https://doi.org/10.3390/en16145264 \
License: ... \
Github: ...


### Initialisation


In [ ]:
# making things autorelad - needed for Jupyter Kernel
%load_ext autoreload
%autoreload 2
%matplotlib widget

# Define the right-path
import sys
import os
folder_path = '../'
os.chdir(folder_path)
sys.path.append(os.getcwd())

# Importing modules
from src.initialisation import load_surfplan, pulley_connectivity
from src.initialisation.input_classes import input_VSM , input_bridle_aero, input_structural_solver
from src.initialisation import input_particleSystem,particles_with_rotational_resistance
from src.particleSystem.ParticleSystem import ParticleSystem
from src.coupling import coupling_aero2struc, coupling_struc2aero
from src.structural import structural_model, structural_mesher
from src.solver import solver_main, solver_utils
from src.post_processing import functions_print, functions_plot, post_processing_utils, post_processing_main
from src.initialisation import actuation_relations
from src.aerodynamic import VSM, breukels_2D, plate_aero, bridle_line_system_aero, tether_aero
from src.initialisation.yaml_loader import config

# from test import test_main
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.optimize
import yaml
import importlib
import pytest
import pandas as pd
import dill
from IPython.display import display, Latex # for pretty printing

## Mutable variables
# Initializing Mutable Variables
points = config.kite.points_ini
# defining vel_app (vector of vel_app_norm)
vel_app = config.vel_wind - config.vel_kite

# Should be the same for each kite
(
    connectivity_matrix,
    wing_connectivity,
) = input_particleSystem.define_connectivity_matrix(config)
params_dict = input_particleSystem.define_params(
    config, wing_connectivity, connectivity_matrix
)
initial_conditions = input_particleSystem.define_initial_conditions_kite(
    config)
points_between_dict = particles_with_rotational_resistance.extract_points_between_dict(
    config
)
if config.is_with_initial_plot:
    functions_plot.plot_initial_geometry(config, points_between_dict)

is_with_rotational_resistance = False
if config.kite_name == "V9_60C":
    is_with_rotational_resistance = True

if is_with_rotational_resistance:
    (
        leadingedge_rotational_resistance_dict,
        strut_rotational_resistance_dict,
    ) = particles_with_rotational_resistance.extract_rotational_resistances_dicts(
        points_between_dict, config
    )
    # first do the struts
    k_bend_strut = 1e10
    params_dict = particles_with_rotational_resistance.initialize_bending_spring(
        k_bend_strut,
        initial_conditions,
        params_dict,
        connectivity_matrix,
        strut_rotational_resistance_dict,
    )
    # secondly do the leading-edge
    k_bend_leadingedge = 1e4
    params_dict = particles_with_rotational_resistance.initialize_bending_spring(
        k_bend_leadingedge,
        initial_conditions,
        params_dict,
        connectivity_matrix,
        leadingedge_rotational_resistance_dict,
    )
# Should be the same for each kite
psystem = ParticleSystem(connectivity_matrix, initial_conditions, params_dict)

# Static-aero


In [ ]:
# Struc --> aero
(
    points_wing_segment_corners_aero_orderded,
    index_transformation_struc_to_aero,
) = coupling_struc2aero.extract_wingpanel_corners_aero_orderded(
    points, config.kite.connectivity.plate_point_indices
)
# creating a dict with key value pairs, to transform from aero to struc
index_transformation_aero_to_struc_dict = {}
for i, value in enumerate(index_transformation_struc_to_aero):
    index_transformation_aero_to_struc_dict[value] = i

# Struc --> aero
points_wing_segment_corners_aero_orderded = points[index_transformation_struc_to_aero]
# Wing Aerodynamic
(
    force_aero_wing_VSM,
    moment_aero_wing_VSM,
    F_rel,
    ringvec,
    controlpoints,
    wingpanels,
    rings,
    coord_L,
    coord_refined,
) = VSM.calculate_force_aero_wing_VSM(
    points_wing_segment_corners_aero_orderded, vel_app, input_VSM
)
# Aero --> struc
if config.coupling_method == "NN":
    force_aero_wing = coupling_aero2struc.aero2struc_NN(
        config.aero.n_chordwise_aero_nodes,
        wingpanels,
        force_aero_wing_VSM,
        points_wing_segment_corners_aero_orderded,
        index_transformation_aero_to_struc_dict,
        points,
    )
elif config.coupling_method == "MSc_Oriol":
    force_aero_wing = coupling_aero2struc.aero2struc(
        points,
        config.kite.connectivity.wing_ci,
        config.kite.connectivity.wing_cj,
        config.kite.connectivity.plate_point_indices,
        force_aero_wing_VSM,
        moment_aero_wing_VSM,
        ringvec,
        controlpoints,
    )
else:
    raise ValueError("Coupling method not recognized; wrong name or typo")

# Bridle Aerodynamics
if config.is_with_aero_bridle:
    force_aero_bridle = bridle_line_system_aero.calculate_force_aero_bridle_thedens2022(
        points, vel_app, input_bridle_aero
    )
else:
    force_aero_bridle = [0]
force_aero = force_aero_wing + force_aero_bridle

# Printing the results
functions_print.print_settings(vel_app, config)
functions_print.print_initial_kite_dimensions(config)
functions_print.print_aero(
    points, vel_app, force_aero_wing, force_aero_bridle, config)

# plotting
post_processing_main.plot(
    [
        [wingpanels, controlpoints, rings, coord_L, F_rel],
        config.kite.wing_rest_lengths_initial,
        config.kite.bridle_rest_lengths_initial,
    ],
    points,
    vel_app,
    config,
)